In [30]:
from main import main
from utils import LoadData, LoadModel, load_data
import argparse
import numpy as np
import time
import copy
import os
import tempfile
from tabulate import tabulate 

import pandas as pd
# pyomo for optimization
import pyomo.environ as pyo

# pytorch for training neural network
import torch.onnx
from torch import nn, optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms

# omlt for interfacing our neural network with pyomo
import onnx
from omlt import OffsetScaling, OmltBlock
from omlt.io.onnx import (
    load_onnx_neural_network_with_bounds,
    write_onnx_model_with_bounds,
    load_onnx_neural_network,
)
from omlt.neuralnet import (FullSpaceNNFormulation, 
    ReluComplementarityFormulation, 
    ReluPartitionFormulation,
    ReducedSpaceNNFormulation)

In [31]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [36]:
def add_arguments(model,model_id,system,scenario):    
    if system == 'membrane':
        args = argparse.Namespace(
            input_dim=7,
            hidden_dim=32,
            hidden_num=2,
            z0_dim=8,
            optimizer='adam',
            epochs=1000,
            batch_size=16,
            lr=1e-4,
            mu=1,
            max_subiter=500,
            eta=0.8,
            sigma=2,
            mu_safe=1e+9,
            dtype=32,
            dataset_path='/home/andresfel9403/KKThNN/KKThPINN/benchmark_membrane.csv',
            val_ratio=0.2,
            job='experiment',
            runs=10)
    else:
        pass

    args.model = model
    args.model_id = model_id
    args.dataset_type = system
    args.scenario = scenario
        
    
    if args.model == 'NN':
        args.loss_type = 'MSE'
    elif args.model == 'PINN':
        args.loss_type = 'PINN'
    elif args.model == 'KKThPINN':
        args.loss_type = 'MSE'
    elif args.model == 'AugLagNN':
        args.loss_type = 'MSE'
    elif args.model == 'ECNN':
        args.loss_type = 'MSE'
    return args

In [37]:
#columns = inputs+outputs

#df = pd.read_csv("benchmark_membrane.csv", usecols = columns)


## Neural Network Training  

In [38]:
args_KKT = add_arguments('KKThPINN','MembraneT_KKThPINN','membrane','demonstration')

In [ ]:
main(args_KKT)



Running NN at run 0
type of A: torch.float32, type of B: torch.float32, type of b: torch.float32
train set size: 614, val set size: 204, test set size: 204
Start Training...
epoch: 00050 loss_train: 0.01838 loss_val: 0.01903 violation_train: 14.69423 violation_val: 14.89681
epoch: 00100 loss_train: 0.00451 loss_val: 0.00482 violation_train: 0.95010 violation_val: 1.01573
epoch: 00150 loss_train: 0.00124 loss_val: 0.00172 violation_train: 0.37644 violation_val: 0.40260
epoch: 00200 loss_train: 0.00104 loss_val: 0.00159 violation_train: 0.23282 violation_val: 0.26453
epoch: 00250 loss_train: 0.00093 loss_val: 0.00143 violation_train: 0.22662 violation_val: 0.24069
epoch: 00300 loss_train: 0.00083 loss_val: 0.00130 violation_train: 0.19296 violation_val: 0.20820
epoch: 00350 loss_train: 0.00074 loss_val: 0.00119 violation_train: 0.19128 violation_val: 0.28261
epoch: 00400 loss_train: 0.00065 loss_val: 0.00110 violation_train: 0.13389 violation_val: 0.19938
epoch: 00450 loss_train: 0.000

/home/andresfel9403/KKThNN/KKThPINN/train.py:264: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(PATH)


After defining the arguments depending on the number of inputs, hidden and output layers, we can train the model selecting the Neural Network model (NN or KKThPINN)

In [7]:
dataset, scaling = load_data(args_KKT.dataset_path)
inputs = dataset[:,:7]
outputs = dataset[:,7:]

data = LoadData(args_KKT)
model_KKT = LoadModel(args_KKT, data)
PATH = '/home/andresfel9403/KKThNN/KKThPINN/models/membrane/KKThPINN/0.2/MembraneT_KKThPINN_0.2_0.pth'
checkpoint = torch.load(PATH)
model_KKT.load_state_dict(checkpoint['state_dict'])
#model_NN.eval()



type of A: torch.float32, type of B: torch.float32, type of b: torch.float32
train set size: 614, val set size: 204, test set size: 204


/tmp/ipykernel_497203/2874336590.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(PATH)


<All keys matched successfully>

In [8]:
x_factor = scaling.scale_[:7]
y_factor = scaling.scale_[7:]

print(x_factor)

[1.46831994e+01 1.04111445e+01 1.42231966e+01 9.99584502e+01
 7.26638589e+02 2.69926289e+06 1.76773269e+02]


In [9]:
scaler = scaler = OffsetScaling(
    offset_inputs={i: 0 for i in range(len(x_factor))},
    factor_inputs={i: x_factor[i] for i in range(len(x_factor))},
    offset_outputs={i: 0 for i in range(len(y_factor))},
    factor_outputs={i: y_factor[i] for i in range(len(y_factor))},
)

In [10]:
x_dummy = torch.from_numpy(inputs[0]).float()
ub = np.max(inputs, 0)
lb = np.min(inputs, 0)

scaled_input_bounds = {i: (lb[i], ub[i]) for i in range(len(inputs[0]))}
print(ub.dtype)

float64


In [11]:
def transfer_weights(original_model, modified_model):
    with torch.no_grad():
        for original_layer, modified_layer in zip(original_model.layers, modified_model.layers):
            if isinstance(original_layer, nn.Linear) and isinstance(modified_layer, nn.Linear):
                modified_layer.weight.copy_(original_layer.weight)
                modified_layer.bias.copy_(original_layer.bias)

In [12]:
def _create_onnx_model(data,args,model,file_path,x,input_bounds):
    args.model='NN'
    print('Saving standard model')
    modified_model = LoadModel(args,data)
    transfer_weights(model,modified_model)
    create_onnx_model(data,modified_model,file_path,x,input_bounds)

In [13]:
def create_onnx_model(data,model,file_path,x,input_bounds):
    
    torch.onnx.export(
        model,
        x,
        file_path,
        input_names=["input"],
        dynamo = False
    )
    write_onnx_model_with_bounds(file_path, None, input_bounds)
    print(f"Wrote PyTorch Onnx model to {file_path}")

In [14]:
_create_onnx_model(data,args_KKT,model_KKT,'KKThPINN_Membrane.onnx',x_dummy,scaled_input_bounds)

Saving standard model
Wrote PyTorch Onnx model to KKThPINN_Membrane.onnx


In [15]:
network_definition_NN = load_onnx_neural_network_with_bounds('KKThPINN_Membrane.onnx') 


In [16]:
formulation_NN = FullSpaceNNFormulation(network_definition_NN)

In [17]:
for layer_id, layer in enumerate(network_definition_NN.layers):
    print(f"{layer_id}\t{layer}\t{layer.activation}")

0	InputLayer(input_size=[7], output_size=[7])	linear
1	DenseLayer(input_size=[7], output_size=[32])	relu
2	DenseLayer(input_size=[32], output_size=[32])	relu
3	DenseLayer(input_size=[32], output_size=[8])	linear


In [18]:
inputs = ['F_MeOH',
    'F_DME',
    'F_H2O',
    'F_N2',
    'T_in',
    'P_in',
    'T_permeate']

outputs = [
    'F_MeOH_O',
    'F_DME_O',
    'F_H2O_O',
    'F_H2O_M_O',
    'F_N2_O',
    'T_Out',
    'T_Out_M',
    'P_Out'
]
model = pyo.ConcreteModel()

# create an OMLT block for the neural network and build its formulation
model.membrane= OmltBlock()
model.membrane.build_formulation(formulation_NN)
model.membrane.inputs[0]

for i in range(len(inputs)):
    model.membrane.inputs[i] = x_dummy[i].item()
    print(pyo.value(model.membrane.inputs[i]))
    print(x_dummy[i].item())
    model.membrane.inputs[i].setlb(lb[i])
    model.membrane.inputs[i].setub(ub[i])

for i in range(len(outputs)):
    model.membrane.outputs[i] = 0.0
    model.membrane.outputs[i].setlb(0.0)

WARNING (W1002): Setting Var 'membrane.scaled_inputs[0]' to a numeric value
`0` outside the bounds (0.002303625455624813, 1.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'membrane.scaled_inputs[0]' to a numeric value
`0` outside the bounds (0.002303625455624813, 1.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'membrane.scaled_inputs[1]' to a numeric value
`0` outside the bounds (0.00045880834803283796, 1.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'membrane.scaled_inputs[2]' to a numeric value
`0` outside the bounds (0.0022512226291679363, 1.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'membrane.scaled_inputs[3]' to a numeric value
`0` outside the bounds (0.10018982303101381, 1.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING 

In [19]:
Tin_idx = inputs.index('T_in')
Tm_idx = inputs.index('T_permeate')
F_MeOH_idx = inputs.index('F_MeOH')
F_DME_idx = inputs.index('F_DME')
F_H2O_idx = inputs.index('F_H2O')
F_N2_idx = inputs.index('F_N2')
P_in_idx = inputs.index('P_in')

FMeOh_O_idx = outputs.index('F_MeOH_O')
FDME_O_idx = outputs.index('F_DME_O')
FH2O_O_idx = outputs.index('F_H2O_O')
FH2O_M_O_idx = outputs.index('F_H2O_M_O')
FN2_O_idx = outputs.index('F_N2_O')
TOut_idx = outputs.index('T_Out')
TOut_M_idx = outputs.index('T_Out_M')
POut_idx = outputs.index('P_Out')

def Tin_bounds(m):
    return (460,m.membrane.inputs[Tin_idx]*x_factor[Tin_idx],1000)

def Tout_bounds(m):
    return (460,y_factor[TOut_idx]*m.membrane.outputs[TOut_idx],1000)

def TOut_mem_bounds(m):
    return (283.15,y_factor[TOut_M_idx]*m.membrane.outputs[TOut_M_idx],450)

def T_mem_bounds(m):
    return (283.15,m.membrane.inputs[Tm_idx]*x_factor[Tm_idx],450)

def TFlow_bounds(m):
    FMeOh = m.membrane.inputs[F_MeOH_idx]*x_factor[F_MeOH_idx]
    FDME = m.membrane.inputs[F_DME_idx]*x_factor[F_DME_idx]
    FH2O = m.membrane.inputs[F_H2O_idx]*x_factor[F_H2O_idx]
    FN2 = m.membrane.inputs[F_N2_idx]*x_factor[F_N2_idx]
    Ftot = FMeOh + FDME + FH2O + FN2
    return (1, Ftot, 100)


def P_bounds(m):
    return (13e5, m.membrane.inputs[P_in_idx]*x_factor[P_in_idx], 27e5)

def POut_bounds(m):
    return (13e5, m.membrane.outputs[POut_idx]*y_factor[POut_idx], 27e5)

def Mas_Bal(m):
    FMeOh = m.membrane.inputs[F_MeOH_idx]*x_factor[F_MeOH_idx]
    FDME = m.membrane.inputs[F_DME_idx]*x_factor[F_DME_idx]
    FH2O = m.membrane.inputs[F_H2O_idx]*x_factor[F_H2O_idx]
    FN2 = m.membrane.inputs[F_N2_idx]*x_factor[F_N2_idx]
    FMeOh_O = m.membrane.outputs[FMeOh_O_idx]*y_factor[FMeOh_O_idx]
    FDME_O = m.membrane.outputs[FDME_O_idx]*y_factor[FDME_O_idx]
    FH2O_O = m.membrane.outputs[FH2O_O_idx]*y_factor[FH2O_O_idx]
    FN2_O = m.membrane.outputs[FN2_O_idx]*y_factor[FN2_O_idx]
    FH2O_M_O = m.membrane.outputs[FH2O_M_O_idx]*y_factor[FH2O_M_O_idx]

    In = FMeOh + FDME + FH2O + FN2
    Out = FMeOh_O + FDME_O + FH2O_O + FN2_O + FH2O_M_O
    MassBal = abs(In - Out)
    return (0, MassBal, 1e-6)

def Flow_Nitrog(m):
    FN2 = m.membrane.inputs[F_N2_idx]*x_factor[F_N2_idx]
    FN2_O = m.membrane.outputs[FN2_O_idx]*y_factor[FN2_O_idx]
    
    return (FN2-FN2_O == 0)

def Flow_MEOH(m):
    return m.membrane.inputs[F_MeOH_idx]*x_factor[F_MeOH_idx] == 0.93*15.58 

def Flow_DME(m):
    return m.membrane.inputs[F_DME_idx]*x_factor[F_DME_idx] == 0.06*15.58

def Flow_H2O(m):
    return m.membrane.inputs[F_H2O_idx]*x_factor[F_H2O_idx] == 0.01*15.58



model.Tin_constr = pyo.Constraint(rule = Tin_bounds)
model.Tm_constr = pyo.Constraint(rule = T_mem_bounds)
model.TFlow_constr = pyo.Constraint(rule = TFlow_bounds)
model.P_constr = pyo.Constraint(rule = P_bounds)
model.MassBal = pyo.Constraint(rule = Mas_Bal)
model.Flow_Nitrog = pyo.Constraint(rule = Flow_Nitrog)
model.Tout_constr = pyo.Constraint(rule = Tout_bounds)
model.Tout_M_constr = pyo.Constraint(rule = TOut_mem_bounds)

model.Flow_MEOH = pyo.Constraint(rule = Flow_MEOH)
model.Flow_DME = pyo.Constraint(rule = Flow_DME)
model.Flow_H2O = pyo.Constraint(rule = Flow_H2O)

model.POut_constr = pyo.Constraint(rule = POut_bounds)




In [20]:
def objective_rule(m):
    prices = {
        "MeOH": 891*1e-6,  # Cost of Methanol in $/kg
        "DME": 2021.84*1e-6,   # Cost of Dimethyl Ether in $/kg
        "H2O": 0.29*1e-6,  # Cost of Water in $/kg
        "N2": 121.254*1e-6  # Cost of Nitrogen in $/kg
    }

    MW = {
        "MeOH": 32.04,  # MW of Methanol in g/mol
        "DME": 46.069,   # Cost of Dimethyl Ether in $/kg
        "H2O": 18.0152,  # Cost of Water in $/kg
        "N2": 28.0134  # Cost of Nitrogen in $/kg
    }

    DME_O = m.membrane.outputs[outputs.index("F_DME_O")]*y_factor[outputs.index("F_DME_O")]
    H2O_O = m.membrane.outputs[outputs.index("F_H2O_O")]*y_factor[outputs.index("F_H2O_O")]
    H2O_M_O = m.membrane.outputs[outputs.index("F_H2O_M_O")]*y_factor[outputs.index("F_H2O_M_O")]
    MeOH_In = m.membrane.inputs[inputs.index("F_MeOH")]*x_factor[inputs.index("F_MeOH")]
    N2_In = m.membrane.inputs[inputs.index("F_N2")]*x_factor[inputs.index("F_N2")]
    DME_In = m.membrane.inputs[inputs.index("F_DME")]*x_factor[inputs.index("F_DME")]

    product_profit = prices["DME"]*DME_O * MW["DME"] + prices["H2O"]*(H2O_O + H2O_M_O) * MW["H2O"]

    feedstock_cost = prices["MeOH"]*MeOH_In * MW["MeOH"] + prices["N2"]*N2_In * MW["N2"] + prices["DME"]*DME_O * MW["DME"]
    return product_profit - feedstock_cost

model.objective = pyo.Objective(rule = objective_rule, sense = pyo.maximize)


In [21]:
solver = pyo.SolverFactory("ipopt")
status = solver.solve(model, tee=True)

Ipopt 3.14.6: 
==> Warning: Treating 64 binary and 0 integer variables as continuous.


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     1593
Number of nonzeros in inequality constraint Jacobian.:      523
Number of nonzeros in Lagrangian Hessian.............:       45

Total number of variables............................:      243
                     variables with only lower bounds:        8
                variables with lower and upper bounds:      227
                     variables with only upper bounds:        0
Total 

In [22]:
F_MEOH_IN = pyo.value(model.membrane.inputs[inputs.index("F_MeOH")])*x_factor[inputs.index("F_MeOH")]
F_MEOH_OUT = pyo.value(model.membrane.outputs[outputs.index("F_MeOH_O")])*y_factor[outputs.index("F_MeOH_O")]

F_DEM_IN = pyo.value(model.membrane.inputs[inputs.index("F_DME")])*x_factor[inputs.index("F_DME")]
F_DME_OUT = pyo.value(model.membrane.outputs[outputs.index("F_DME_O")])*y_factor[outputs.index("F_DME_O")]

F_H2O_IN = pyo.value(model.membrane.inputs[inputs.index("F_H2O")])*x_factor[inputs.index("F_H2O")]
F_H2O_OUT = pyo.value(model.membrane.outputs[outputs.index("F_H2O_O")])*y_factor[outputs.index("F_H2O_O")]
F_H2O_M_OUT = pyo.value(model.membrane.outputs[outputs.index("F_H2O_M_O")])*y_factor[outputs.index("F_H2O_M_O")]

F_N2_IN = pyo.value(model.membrane.inputs[inputs.index("F_N2")])*x_factor[inputs.index("F_N2")]
F_N2_OUT = pyo.value(model.membrane.outputs[outputs.index("F_N2_O")])*y_factor[outputs.index("F_N2_O")]

T_IN = pyo.value(model.membrane.inputs[inputs.index("T_in")])*x_factor[inputs.index("T_in")]
T_M_IN = pyo.value(model.membrane.inputs[inputs.index("T_permeate")])*x_factor[inputs.index("T_permeate")]
T_OUT = pyo.value(model.membrane.outputs[outputs.index("T_Out")])*y_factor[outputs.index("T_Out")]
T_M_OUT = pyo.value(model.membrane.outputs[outputs.index("T_Out_M")])*y_factor[outputs.index("T_Out_M")]

P_IN = pyo.value(model.membrane.inputs[inputs.index("P_in")])*x_factor[inputs.index("P_in")]
P_OUT = pyo.value(model.membrane.outputs[outputs.index("P_Out")])*y_factor[outputs.index("P_Out")]

CONVERSION = (F_MEOH_IN - F_MEOH_OUT)/F_MEOH_IN
Mas_Bal = pyo.value(model.MassBal)

data_tabulate = [
    ["Input Parameter","Value","Output Parameter","Value"],
    ["F_MeOH", F_MEOH_IN, "F_MeOH_O", F_MEOH_OUT],
    ["F_DME", F_DEM_IN, "F_DME_O", F_DME_OUT],
    ["F_H2O", F_H2O_IN, "F_H2O_O", F_H2O_OUT],
    ["-", "-", "F_H2O_M_O", F_H2O_M_OUT],
    ["F_N2", F_N2_IN, "F_N2_O", F_N2_OUT],
    ["T_in", T_IN, "T_Out", T_OUT],
    ["P_in", P_IN, "P_Out", P_OUT],
    ["T_In_M", T_M_IN, "T_Out_M", T_M_OUT],
]

conversion_table = [
    ["MeOH Conversion", CONVERSION]
]

print(tabulate(data_tabulate, headers="firstrow", tablefmt="grid", colalign=("left", "left", "left", "left")))
print(tabulate(conversion_table, tablefmt="grid", colalign=("left", "left")))
print(f"Mass Balance Violation: {Mas_Bal}")

+-------------------+--------------------+--------------------+--------------+
| Input Parameter   | Value              | Output Parameter   | Value        |
+===================+====================+====================+==============+
| F_MeOH            | 14.489399999999817 | F_MeOH_O           | 0.993486     |
+-------------------+--------------------+--------------------+--------------+
| F_DME             | 0.9347999999998364 | F_DME_O            | 7.19134      |
+-------------------+--------------------+--------------------+--------------+
| F_H2O             | 0.1557999999998801 | F_H2O_O            | -5.55737e-08 |
+-------------------+--------------------+--------------------+--------------+
| -                 | -                  | F_H2O_M_O          | 7.39517      |
+-------------------+--------------------+--------------------+--------------+
| F_N2              | 10.014818440954203 | F_N2_O             | 10.0148      |
+-------------------+--------------------+----------

In [23]:
print(x_factor)

[1.46831994e+01 1.04111445e+01 1.42231966e+01 9.99584502e+01
 7.26638589e+02 2.69926289e+06 1.76773269e+02]
